In [32]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import os
import pandas as pd
import sqlite3
import sqlalchemy 
from bs4 import Comment,BeautifulSoup
import requests

from datetime import date
from dateutil.rrule import rrule, DAILY

#Standard Imports
import csv
import pandas as pd
import numpy as np
from pandas import DataFrame,Series
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold
import re
from sklearn.cluster import KMeans
import random
from sklearn import datasets
from sklearn import metrics
from dateutil import rrule
from datetime import datetime

%matplotlib inline

pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option("display.max_columns",50)
pd.set_option("display.max_rows",100)

import matplotlib
#matplotlib.style.use('default')
import seaborn as sns
from datetime import datetime

In [14]:
def soup_coin(coin,start,end,zip_all):
    ticker = zip_all.get(coin)
    
    #This function returns a dataframe of the coin within a time frame
    # e.g df = soup_coin('bitcoin',20180101,20180109)
    url = 'https://coinmarketcap.com/currencies/' + str(coin) + '/historical-data/' + '?start=' + str(start) + '&end=' + str(end)
    print url
    html = requests.get(url).text
    soup = BeautifulSoup(html,'html5lib')

    try:
        table = soup('table')[0] #assume one table
        cols = [inner.renderContents() for inner in soup('table')[0].findAll('th')]
        tds = [el.renderContents() for el in soup('table')[0].findAll('td')]
    except IndexError:
        print "IndexError"
        return DataFrame()

    #Check for empty table
    if tds[0] != "No data was found for the selected time period.":
        df = DataFrame(np.array(tds).reshape((len(tds)/len(cols),len(cols))),columns = cols)
        #Global replacing
        #Replace '-' with None 
        df = df.applymap(lambda x: 0 if x == '-' else x)
        df = df.applymap(lambda x: x.replace(',','') if x else x)
        df.Date = df.Date.apply(lambda x: datetime.strptime(x,'%b %d %Y'))


        #Format columns
        df.Open = df.Open.astype(float)
        df.High = df.High.astype(float)
        df.Low = df.Low.astype(float)
        df.Close = df.Close.astype(float)
        df.Volume = df['Volume'].apply(lambda x: int(x))
        df['Market Cap'] = df['Market Cap'].apply(lambda x: int(x))
        df['Ticker'] = [ticker] * len(df.Open) 
        return df
    else:
        print "No data found for date"
        return DataFrame()

In [34]:
#https://coinmarketcap.com/currencies/deepbrain-chain/historical-data/?start=20130428&end=20180116
df = soup_coin('litecoin',20180119,20180119,dict(zip_all))
df

https://coinmarketcap.com/currencies/litecoin/historical-data/?start=20180119&end=20180119


,Date,Open,High,Low,Close,Volume,Market Cap,Ticker
0,2018-01-19,191.19,202.07,186.94,192.96,560284000,10480800000,LTC


In [7]:
def all_tickers():
    url = 'https://coinmarketcap.com/coins/views/all/'
    url = 'https://coinmarketcap.com/all/views/all/'
    soup = BeautifulSoup(requests.get(url).text,'html5lib')
    all_tickers = [el.renderContents() for el in soup.findAll('td', {'class':"text-left col-symbol"})]
    return all_tickers

In [8]:
all_tickers = all_tickers()

In [9]:
def all_coin_names():
    #This function 
    url = 'https://coinmarketcap.com/coins/views/all/'
    url = 'https://coinmarketcap.com/all/views/all/'

    soup = BeautifulSoup(requests.get(url).text,'html5lib')
    ac = soup.findAll('a', {'class':"currency-name-container"},href=True)
    all_coins = [name['href'].split('/')[-2] for name in ac]
    return all_coins

In [10]:
all_coin_names = all_coin_names()

In [11]:
zip_all = zip(all_coin_names,all_tickers)

In [8]:
def create_all_coin_df(all_coin_names):
    concat_df = DataFrame()
    for coin in all_coin_names:
        try:
            concat_df = pd.concat([concat_df,soup_coin(coin,20171225,20180115,dict(zip_all))])
        except ValueError:
            print "Error with coin", coin
            continue
    return concat_df

# Function to create DB from scratch

In [21]:
def create_all_coin_df(all_coin_names,start,end,zip_all):
    #TODO: Need to add check for new coins that are added to CMC
    conn = sqlite3.connect('/Users/nathanchoo/cryptodb/crypto.db')
    cur = conn.cursor()
    
    #cur.execute('''DROP TABLE IF EXISTS DAILY''')
    #cur.execute('''CREATE TABLE DAILY ( id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE, \
    #              date TEXT, open NUMERIC, high NUMERIC, low NUMERIC, close NUMERIC, volume INTEGER, marketcap INTEGER, ticker TEXT)''')
    
    for i,coin in enumerate(all_coin_names):
        print i,coin
        df = soup_coin(coin,start,end,dict(zip_all))
        if not df.empty:
            try:
                for i,row in df.iterrows():
                    cur.execute("INSERT INTO DAILY (date,open,high,low,close,volume,marketcap,ticker) VALUES (?,?,?,?,?,?,?,?)",\
                                (str(row['Date']),row['Open'],row['High'],row['Low'],row['Close'],row['Volume'],row['Market Cap'],row['Ticker']));
                    conn.commit()
            except ValueError:
                print "Error with ", coin
                continue
        else:
            print "Error with (empty table):", coin
    cur.close()


In [47]:
#create_all_coin_df(all_coin_names,20130428,20180115,zip_all)

# FUNCTION TO ADD NEW COINS FROM CMC

In [13]:
def add_new_coins_df(all_coin_names,all_tickers,start,end,zip_all):
    start = start
    end = end
    #TODO: Need to add check for new coins that are added to CMC
    conn = sqlite3.connect('/Users/nathanchoo/cryptodb/crypto.db')
    cur = conn.cursor()
    cur.execute('''SELECT DISTINCT(Ticker) FROM DAILY''')
    
    d = dict([(i[1],i[0]) for i in zip_all])
    all_ticker_in_db = cur.fetchall()
    s2 = set(all_tickers)
    s1 = set([i[0] for i in all_ticker_in_db])
    outstanding_new_coins = list(s2.difference(s1))
    print outstanding_new_coins
    
    for i,coin in enumerate(outstanding_new_coins):
        print i,coin
        df = soup_coin(d.get(coin),start,end,dict(zip_all))
        if not df.empty:
            try:
                for i,row in df.iterrows():
                    cur.execute("INSERT INTO DAILY (date,open,high,low,close,volume,marketcap,ticker) VALUES (?,?,?,?,?,?,?,?)",\
                                (str(row['Date']),row['Open'],row['High'],row['Low'],row['Close'],row['Volume'],row['Market Cap'],row['Ticker']));
                    conn.commit()
            except ValueError:
                print "Error with ", coin
                continue
        else:
            print "Error with (empty table):", coin
    cur.close()

In [9]:
#add_new_coins_df(all_coin_names,all_tickers,20130428,20180118,zip_all)

# FUNCTION TO UPDATE DB OF EXISTING COINS

In [15]:
def update_coins_df(start,end,zip_all):
    #TODO: Need to add check for new coins that are added to CMC
    d = dict([(i[1],i[0]) for i in zip_all])
    conn = sqlite3.connect('/Users/nathanchoo/cryptodb/crypto.db')
    cur = conn.cursor()    
    cur.execute('''SELECT DISTINCT(Ticker) FROM DAILY''')
    all_ticker_in_db = cur.fetchall()
    all_ticker_in_db = [i[0] for i in all_ticker_in_db]
    
    for i,coin in enumerate(all_ticker_in_db):
        print i,coin
        df = soup_coin(d.get(coin),start,end,dict(zip_all))
        if not df.empty:
            try:
                for i,row in df.iterrows():
                    cur.execute("INSERT INTO DAILY (date,open,high,low,close,volume,marketcap,ticker) VALUES (?,?,?,?,?,?,?,?)",\
                                (str(row['Date']),row['Open'],row['High'],row['Low'],row['Close'],row['Volume'],row['Market Cap'],row['Ticker']));
                    conn.commit()
            except ValueError:
                print "Error with ", coin
                continue
        else:
            print "Error with (empty table):", coin
    cur.close()

In [33]:
update_coins_df(20180204,20180207,zip_all)

0 BTC
https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20180204&end=20180207
1 ETH
https://coinmarketcap.com/currencies/ethereum/historical-data/?start=20180204&end=20180207
2 XRP
https://coinmarketcap.com/currencies/ripple/historical-data/?start=20180204&end=20180207
3 BCH
https://coinmarketcap.com/currencies/bitcoin-cash/historical-data/?start=20180204&end=20180207
4 ADA
https://coinmarketcap.com/currencies/cardano/historical-data/?start=20180204&end=20180207
5 LTC
https://coinmarketcap.com/currencies/litecoin/historical-data/?start=20180204&end=20180207
6 NEO
https://coinmarketcap.com/currencies/neo/historical-data/?start=20180204&end=20180207
7 XEM
https://coinmarketcap.com/currencies/nem/historical-data/?start=20180204&end=20180207
8 XLM
https://coinmarketcap.com/currencies/stellar/historical-data/?start=20180204&end=20180207
9 MIOTA
https://coinmarketcap.com/currencies/iota/historical-data/?start=20180204&end=20180207
10 EOS
https://coinmarketcap.com/currencies

# QUERY

In [24]:
conn = sqlite3.connect('/Users/nathanchoo/cryptodb/crypto.db')
cur = conn.cursor()    
cur.execute('''SELECT * FROM Daily WHERE date = '2018-01-01 00:00:00' ORDER BY marketcap desc limit 200''')
q = cur.fetchall()

In [17]:
def query_mc_by_date(start,end,limit,switch):
    '''Function is used to query market cap by date on Daily table. Returns dataframe'''
    
    ed = {}
    date_list = []
    start = start
    end = end

    #print start
    #print end
    for dt in rrule.rrule(rrule.DAILY,
                          dtstart=datetime.strptime(start, '%Y-%m-%d %H:%M:%S'),
                          until=datetime.strptime(end, '%Y-%m-%d %H:%M:%S')):
        date_list.append(dt.strftime('%Y-%m-%d %H:%M:%S'))
        
    conn = sqlite3.connect('/Users/nathanchoo/cryptodb/crypto.db')
    cur = conn.cursor()    
    
    for date in date_list:
        print date
        query_statement = '''SELECT * FROM Daily WHERE date = ''' + "'" + str(date) + "'"  + ''' ORDER BY marketcap desc limit ''' + str(limit)
        print query_statement
        cur.execute(query_statement)
        q = cur.fetchall()
        if switch == 'Name':
            ed[q[0][1].split(' ')[0]] = [i[-1] for i in q]
        else:
            ed[q[0][1].split(' ')[0]] = [(i[-1],i[-2]) for i in q]

    
    return ed

In [26]:
d1 = query_mc_by_date('2018-01-15 00:00:00','2018-02-03 00:00:00',1000,"Name")

2018-01-15 00:00:00
SELECT * FROM Daily WHERE date = '2018-01-15 00:00:00' ORDER BY marketcap desc limit 1000
2018-01-16 00:00:00
SELECT * FROM Daily WHERE date = '2018-01-16 00:00:00' ORDER BY marketcap desc limit 1000
2018-01-17 00:00:00
SELECT * FROM Daily WHERE date = '2018-01-17 00:00:00' ORDER BY marketcap desc limit 1000
2018-01-18 00:00:00
SELECT * FROM Daily WHERE date = '2018-01-18 00:00:00' ORDER BY marketcap desc limit 1000
2018-01-19 00:00:00
SELECT * FROM Daily WHERE date = '2018-01-19 00:00:00' ORDER BY marketcap desc limit 1000
2018-01-20 00:00:00
SELECT * FROM Daily WHERE date = '2018-01-20 00:00:00' ORDER BY marketcap desc limit 1000
2018-01-21 00:00:00
SELECT * FROM Daily WHERE date = '2018-01-21 00:00:00' ORDER BY marketcap desc limit 1000
2018-01-22 00:00:00
SELECT * FROM Daily WHERE date = '2018-01-22 00:00:00' ORDER BY marketcap desc limit 1000
2018-01-23 00:00:00
SELECT * FROM Daily WHERE date = '2018-01-23 00:00:00' ORDER BY marketcap desc limit 1000
2018-01-24

In [27]:
d1_x = DataFrame(d1)

In [29]:
d2 = query_mc_by_date('2018-01-15 00:00:00','2018-02-03 00:00:00',1000,"")

2018-01-15 00:00:00
SELECT * FROM Daily WHERE date = '2018-01-15 00:00:00' ORDER BY marketcap desc limit 1000
2018-01-16 00:00:00
SELECT * FROM Daily WHERE date = '2018-01-16 00:00:00' ORDER BY marketcap desc limit 1000
2018-01-17 00:00:00
SELECT * FROM Daily WHERE date = '2018-01-17 00:00:00' ORDER BY marketcap desc limit 1000
2018-01-18 00:00:00
SELECT * FROM Daily WHERE date = '2018-01-18 00:00:00' ORDER BY marketcap desc limit 1000
2018-01-19 00:00:00
SELECT * FROM Daily WHERE date = '2018-01-19 00:00:00' ORDER BY marketcap desc limit 1000
2018-01-20 00:00:00
SELECT * FROM Daily WHERE date = '2018-01-20 00:00:00' ORDER BY marketcap desc limit 1000
2018-01-21 00:00:00
SELECT * FROM Daily WHERE date = '2018-01-21 00:00:00' ORDER BY marketcap desc limit 1000
2018-01-22 00:00:00
SELECT * FROM Daily WHERE date = '2018-01-22 00:00:00' ORDER BY marketcap desc limit 1000
2018-01-23 00:00:00
SELECT * FROM Daily WHERE date = '2018-01-23 00:00:00' ORDER BY marketcap desc limit 1000
2018-01-24

In [30]:
d2_x = DataFrame(d2)

In [31]:
d2_x.to_csv('d2_x.csv')

In [28]:
d1_x.to_csv('d1_x.csv')

In [16]:
os.getcwd()

'/Users/nathanchoo/Desktop/Craig/DAT_SF_10/crypto'

In [16]:
d1_x

,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08,2018-01-09,2018-01-10,2018-01-11,2018-01-12,2018-01-13,2018-01-14,2018-01-15,2018-01-16,2018-01-17,2018-01-18,2018-01-19,2018-01-20,2018-01-21
0,"(BTC, 236725000000)","(BTC, 228579000000)","(BTC, 251312000000)","(BTC, 256250000000)","(BTC, 259748000000)","(BTC, 293091000000)","(BTC, 294222000000)","(BTC, 276612000000)","(BTC, 253935000000)","(BTC, 244981000000)","(BTC, 251387000000)","(BTC, 225986000000)","(BTC, 234391000000)","(BTC, 241447000000)","(BTC, 231334000000)","(BTC, 232517000000)","(BTC, 192123000000)","(BTC, 188242000000)","(BTC, 192150000000)","(BTC, 195979000000)","(BTC, 216740000000)"
1,"(XRP, 88945900000)","(XRP, 91791800000)","(XRP, 95457200000)","(XRP, 120763000000)","(XRP, 127870000000)","(XRP, 117033000000)","(XRP, 119819000000)","(XRP, 130302000000)","(ETH, 110999000000)","(ETH, 125976000000)","(ETH, 122879000000)","(ETH, 112263000000)","(ETH, 123163000000)","(ETH, 135503000000)","(ETH, 132404000000)","(ETH, 125389000000)","(ETH, 102974000000)","(ETH, 98642100000)","(ETH, 99863900000)","(ETH, 101451000000)","(ETH, 112226000000)"
2,"(ETH, 73075800000)","(ETH, 74695700000)","(ETH, 85703500000)","(ETH, 93049400000)","(ETH, 94423900000)","(ETH, 96326500000)","(ETH, 100981000000)","(ETH, 112164000000)","(XRP, 94657500000)","(XRP, 82511000000)","(XRP, 76711800000)","(XRP, 75172500000)","(XRP, 79469700000)","(XRP, 78195800000)","(XRP, 71909400000)","(XRP, 65018400000)","(XRP, 45854700000)","(XRP, 50577100000)","(XRP, 61291900000)","(XRP, 60543000000)","(XRP, 61061900000)"
3,"(BCH, 42804800000)","(BCH, 41114800000)","(BCH, 45613300000)","(BCH, 44069100000)","(BCH, 40557600000)","(BCH, 43653200000)","(BCH, 47054000000)","(BCH, 47091600000)","(BCH, 40770900000)","(BCH, 40398000000)","(BCH, 48880800000)","(BCH, 41859100000)","(BCH, 44260300000)","(BCH, 45439500000)","(BCH, 42983500000)","(BCH, 40708800000)","(BCH, 29899500000)","(BCH, 29643600000)","(BCH, 29442200000)","(BCH, 30191500000)","(BCH, 34415500000)"
4,"(ADA, 18637600000)","(ADA, 18788700000)","(ADA, 20214800000)","(ADA, 28364900000)","(ADA, 30364400000)","(ADA, 25849300000)","(ADA, 26553200000)","(ADA, 25909700000)","(ADA, 23287300000)","(ADA, 21153200000)","(ADA, 20521800000)","(ADA, 18039200000)","(ADA, 23785800000)","(ADA, 22866700000)","(ADA, 20631800000)","(ADA, 20239200000)","(ADA, 16181700000)","(ADA, 16367600000)","(ADA, 16840800000)","(ADA, 17022800000)","(ADA, 18314600000)"
5,"(LTC, 12639700000)","(LTC, 12496900000)","(LTC, 13957800000)","(XEM, 16520300000)","(XEM, 15195600000)","(TRX, 14760400000)","(LTC, 16178100000)","(XEM, 16454000000)","(XEM, 15047200000)","(XEM, 13913900000)","(LTC, 13758400000)","(LTC, 12572900000)","(XEM, 13127300000)","(LTC, 14265000000)","(LTC, 12993700000)","(LTC, 12752100000)","(LTC, 10417800000)","(LTC, 10245500000)","(LTC, 10480800000)","(LTC, 10658300000)","(LTC, 11585700000)"
6,"(MIOTA, 9883750000)","(MIOTA, 11363400000)","(MIOTA, 11109700000)","(XLM, 15953600000)","(TRX, 13779500000)","(XEM, 14216700000)","(XEM, 14898800000)","(LTC, 15726800000)","(LTC, 13920200000)","(LTC, 13566300000)","(XEM, 13043500000)","(XEM, 11824500000)","(LTC, 12964300000)","(XEM, 13590300000)","(XEM, 12488300000)","(XEM, 12460300000)","(NEO, 9407150000)","(NEO, 9269200000)","(XEM, 9645220000)","(XEM, 9901720000)","(XEM, 11019300000)"
7,"(XEM, 9315310000)","(XEM, 9374970000)","(XEM, 10333700000)","(LTC, 13403400000)","(XLM, 13219400000)","(LTC, 13645300000)","(XLM, 12688400000)","(TRX, 13491200000)","(XLM, 11229100000)","(XLM, 10523600000)","(MIOTA, 10496300000)","(XLM, 9998070000)","(XLM, 12198400000)","(XLM, 12025900000)","(XLM, 11190600000)","(NEO, 12217900000)","(XEM, 8788260000)","(XEM, 9234340000)","(NEO, 9181130000)","(NEO, 9132020000)","(NEO, 9764800000)"
8,"(DASH, 8207990000)","(XLM, 8873700000)","(XLM, 10059800000)","(MIOTA, 11466400000)","(LTC, 13164800000)","(XLM, 11854000000)","(TRX, 11419000000)","(XLM, 12442100000)","(MIOTA, 1068050000

In [62]:
from __future__ import division

In [52]:
for i in range(1,len(d1_x.columns)):
    head = d1_x.columns[i]
    col = d1_x.iloc[:,i]
    

In [170]:
col1 = d1_x.iloc[:,0]
col2 = d1_x.iloc[:,1]
col8 = d1_x.iloc[:,8]

In [163]:
#def col_comp(col1,col2):
print len(col2)
print len(col1)

1200
1200


In [164]:
len(col3.unique())

1196

In [108]:
def run(col1,col2):
    tmp_list = []
    for i in col2:
        match = False
        for j in col1:
            if i[0] == j[0]:
                if j[1] == 0:
                    percent_growth = 0
                    match = True
                    break
                else:
                    percent_growth = (i[1]-j[1])/j[1]
                    tmp_list.append((i[0],percent_growth))
                    match = True
                    break
        if match == False:
            tmp_list.append((i[0],'N/A'))
            break
    return tmp_list

In [155]:
t = run(col1,col2)

1071

In [114]:
s1 = set([i[0] for i in col1])

In [115]:
s2 = set([i[0] for i in col2])

In [116]:
len(s1.intersection(s2))

1178

In [172]:
col8[col8.duplicated()]

196     (PRL, 110375000)
197     (PRL, 110375000)
1112            (CMS, 0)
1142            (XIN, 0)
1184            (WIC, 0)
1199            (XID, 0)
Name: 2018-01-09, dtype: object

In [171]:
import collections
print [item for item, count in collections.Counter(col8).items() if count > 1]

[(u'PRL', 110375000), (u'XIN', 0), (u'XID', 0), (u'WIC', 0), (u'CMS', 0)]


In [177]:
l8 =  [i[0] for i in col8]

In [186]:
l8.index('PRL')

195

In [184]:
d1_x.ix[d1_x.ix[:,8] == "('PRL', 110375000)",8]

Series([], Name: 2018-01-09, dtype: object)

In [200]:
[z for z in col1 if z == 'PRL']

[]